In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# KFP Pipeline Compile and Run

<table align="left">
  <td>
    <a href="https://console.cloud.google.com/ai-platform/notebooks/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/fraudfinder/raw/main/06_model_training_pipeline.ipynb">
       <img src="https://www.gstatic.com/cloud/images/navigation/vertex-ai.svg" alt="Google Cloud Notebooks">Open in Cloud Notebook
    </a>
  </td> 
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/fraudfinder/blob/main/06_model_training_pipeline.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Open in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/fraudfinder/blob/main/06_model_training_pipeline.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
</table>

## Overview

[Fraudfinder](https://github.com/googlecloudplatform/fraudfinder) is a series of labs on how to build a real-time fraud detection system on Google Cloud. Throughout the Fraudfinder labs, you will run the Kubeflow Pipeline as a modular compononets. The main purpose of this notebook is testing the pipeline as code. You can skip this notebook if you sure the pipeline code works properly.

### Objective

This notebook shows how to compile and run modular components of a pipeline. You need the modular compononet, to be used in MLOPS process. 

This lab uses the following Google Cloud services and resources:

- [Vertex AI](https://cloud.google.com/vertex-ai/)
- [BigQuery](https://cloud.google.com/bigquery/)

Steps performed in this notebook:

    * Compile the KFP Pipeline
    * Run the KFP Pipeline

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* BigQuery

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing), [BigQuery pricing](https://cloud.google.com/bigquery/pricing) and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

### Load configuration settings from the setup notebook

Set the constants used in this notebook and load the config settings from the `00_environment_setup.ipynb` notebook.

In [ ]:
GCP_PROJECTS = !gcloud config get-value project
PROJECT_ID = GCP_PROJECTS[0]
BUCKET_NAME = f"{PROJECT_ID}-fraudfinder"
config = !gsutil cat gs://{BUCKET_NAME}/config/notebook_env.py
print(config.n)
exec(config.n)

### Import libraries and define constants

#### Libraries

In [ ]:
import os
import random
from datetime import datetime

In [ ]:
MODEL_DISPLAY_NAME = f'xgboost-fruad-finder'
PIPELINE_NAME = f'fraud-finder-xgb-pipeline-{ID}'
PIPELINE_STORE_URI = f"{BUCKET_NAME}/pipeline-store/"
IMAGE_REPOSITORY = f'fraudfinder-{ID}'
IMAGE_NAME='dask-xgb-classificator'
IMAGE_TAG='v1'
IMAGE_URI=f"us-central1-docker.pkg.dev/{PROJECT_ID}/{IMAGE_REPOSITORY}/{IMAGE_NAME}:{IMAGE_TAG}"

os.environ["PROJECT_ID"] = PROJECT_ID

In [ ]:
!mkdir ./pipelines/components/

## Compile your pipeline into a JSON file

Please check the `deploy_kfp_pipeline` directory. We already have added the code for building the pipline and its's components in the `pipeine` directory. Please feel free to change it.

After the workflow of your pipeline is defined, you can proceed to compile the pipeline into a JSON format. The JSON file will include all the information for executing your pipeline on Vertex AI Pipelines.

In [ ]:
!python deploy_kfp_pipeline/pipeline_compile.py  --pipeline-name=$PIPELINE_NAME

## Submit your pipeline run
Once the workflow of your pipeline is compiled into the JSON format, you can use the Vertex AI Python client to submit and run your pipeline.

In [ ]:
!gsutil ubla set on gs://{BUCKET_NAME}

In [ ]:
pipelines_file_location = os.path.join('./pipelines/', f'{PIPELINE_NAME}.json')
!python ./deploy_kfp_pipeline/pipeline_run.py --pipelines-file-location=$pipelines_file_location